In [1]:
# mount google drive and export your work
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install faker
!pip install pymongo
from faker import Faker
from pymongo import MongoClient
import random
import datetime
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.3 MB/s eta 0:00:00


# Install MongoDB

In [3]:
!apt-get install gnupg curl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.18).
gnupg is already the newest version (2.2.27-3ubuntu2.1).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
!curl -fsSL https://pgp.mongodb.com/server-7.0.asc | \
   sudo gpg -o /usr/share/keyrings/mongodb-server-7.0.gpg \
   --dearmor

In [5]:
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-7.0.list

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse


In [6]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:3 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release [2,090 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:8 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release.gpg [866 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,648 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,451 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,278 kB]
Ign:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-ba

In [7]:
!apt-get install -y mongodb-org

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org-database mongodb-org-database-tools-extra
  mongodb-org-mongos mongodb-org-server mongodb-org-shell mongodb-org-tools
The following NEW packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org mongodb-org-database
  mongodb-org-database-tools-extra mongodb-org-mongos mongodb-org-server mongodb-org-shell
  mongodb-org-tools
0 upgraded, 9 newly installed, 0 to remove and 52 not upgraded.
Need to get 162 MB of archives.
After this operation, 581 MB of additional disk space will be used.
Get:1 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0/multiverse amd64 mongodb-database-tools amd64 100.10.0 [46.4 MB]
Get:2 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0/multiverse amd64 mongodb-mongosh amd64 2.3.2 [54.1 MB]
Get:3 https://repo.mongodb.org/apt/u

In [9]:
# remove the directory /data and /data/db if it is exist

!rm -rf /data/db
!rm -rf /data

In [10]:
!mkdir /data #create new directory
!mkdir /data/db #create new subdirectory

In [11]:
import subprocess
subprocess.Popen(["mongod"])

<Popen: returncode: None args: ['mongod']>

In [12]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local']

# Task I - Document Database Design and Population (5%)

In [13]:
# Download and unzip
!wget https://github.com/matthewpecsok/data_engineering/raw/main/data/patient_records.zip
!wget https://github.com/matthewpecsok/data_engineering/raw/main/data/medication_database.db
!unzip patient_records.zip

--2024-10-17 01:25:46--  https://github.com/matthewpecsok/data_engineering/raw/main/data/patient_records.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/patient_records.zip [following]
--2024-10-17 01:25:46--  https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/patient_records.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25417183 (24M) [application/zip]
Saving to: ‘patient_records.zip’

patient_records.zip 100%[===================>]  24.24M  --.-KB/s    in 0.1s    

2024-10-17 01:25:47 (196 MB/s) - ‘patient_records.zip’ s

In [14]:
import json
from pymongo import MongoClient
import glob

# Connect to MongoDB
client = MongoClient()
db = client['patient_record_db']  # Replace with your desired database name
collection = db['patients']  # Replace with your desired collection name

# Load and insert data from all matching JSON files
for file in glob.glob('patient_records_batch_*.json'):
    with open(file, 'r') as f:
        data = json.load(f)
        # Insert data into collection
        collection.insert_many(data)
print("Data insertion complete.")

Data insertion complete.


In [15]:
# db.drop_collection() # uncomment this one and run if the list already exist

In [16]:
db.list_collection_names() #list

['patients']

In [17]:
# loop through collections and count the number of documents
for collection_name in db.list_collection_names():
  collection = db.get_collection(collection_name)
  print(f'{collection_name}:{collection.count_documents({})}')

patients:100000


## Query 1: Find first 10 patients

In [18]:
# find the record patients

for patient in collection.find({}).limit(10):
    print(patient)


{'_id': ObjectId('671067a936b62531bc39057e'), 'patient_id': 948910, 'name': 'Chelsea Brady', 'gender': None, 'birth_date': '1988-09-25', 'address': '5943 Diana Gardens\nSouth Gregburgh, MT 88070', 'admits': [{'admit_reason': 'Fever', 'admit_date': '2020-10-01', 'symptoms': 'Ability less smile but piece.', 'provider': 'Dr. Brown', 'discharge_date': '2022-02-17', 'admit_cost': 6460, 'patient_id': 948910}, {'admit_reason': 'Infection', 'admit_date': '2019-12-19', 'symptoms': 'Development almost tax recent receive your fact fill.', 'provider': 'Dr. Taylor', 'discharge_date': '2022-06-19', 'admit_cost': 1867, 'patient_id': 948910}, {'admit_reason': 'Chronic condition', 'admit_date': '2022-09-18', 'symptoms': 'Cover theory would population foreign mention.', 'provider': 'Dr. Parker', 'discharge_date': '2023-10-22', 'admit_cost': 4543, 'patient_id': 948910}, {'admit_reason': 'Neurological problem', 'admit_date': '2022-10-03', 'symptoms': 'Stay wind laugh second government rest.', 'provider': 

## Query 2: query find the patients with gender

In [19]:
# query find the patients with gender

for patient in collection.find({"gender": "Female"}).limit(10):
    print(patient)

{'_id': ObjectId('671067a936b62531bc39057f'), 'patient_id': 979020, 'name': 'Wendy Wise', 'gender': 'Female', 'birth_date': None, 'address': '055 Debra Landing\nPort Scott, VT 91484', 'admits': [{'admit_reason': 'Respiratory issue', 'admit_date': '2022-07-23', 'symptoms': 'Card question form receive.', 'provider': 'Dr. Wood', 'discharge_date': '2023-05-15', 'admit_cost': 8154, 'patient_id': 979020}, {'admit_reason': 'Injury', 'admit_date': '2023-09-30', 'symptoms': 'Choose six memory particular cell why instead.', 'provider': 'Dr. Brown', 'discharge_date': '2023-12-10', 'admit_cost': 7362, 'patient_id': 979020}, {'admit_reason': 'Heart condition', 'admit_date': '2023-03-21', 'symptoms': 'Professor whole beautiful suddenly loss camera reality create.', 'provider': 'Dr. Clark', 'discharge_date': '2023-06-07', 'admit_cost': 5638, 'patient_id': 979020}, {'admit_reason': 'Allergic reaction', 'admit_date': '2021-07-06', 'symptoms': 'Available ability at discussion hotel watch direction.', 'p

# Task II - Document Data Extraction (25%)

In [52]:
# Query the database (e.g., find all documents)
patient_data = collection.find({}).limit(10)

# Create a list of dictionaries to store the extracted data
data_list = list(patient_data)

# Create a Pandas DataFrame from the list of dictionaries
df_patient = pd.DataFrame(data_list)

# Display the DataFrame
print(df_patient)

                        _id  patient_id             name  gender  birth_date  \
0  671067a936b62531bc39057e      948910    Chelsea Brady    None  1988-09-25   
1  671067a936b62531bc39057f      979020       Wendy Wise  Female        None   
2  671067a936b62531bc390580      108125  Jose Mcmahon MD  Female  1945-07-13   
3  671067a936b62531bc390581      883751    Maurice Scott    None        None   
4  671067a936b62531bc390582      324717    Dana Sullivan    None        None   
5  671067a936b62531bc390583      855133      Keith Perez  Female        None   
6  671067a936b62531bc390584      629891  Elizabeth Reyes    Male        None   
7  671067a936b62531bc390585      721059       Ethan Odom  Female  1937-01-21   
8  671067a936b62531bc390586      386341   Cynthia Miller    None        None   
9  671067a936b62531bc390587      629792  Darrell Perkins    Male        None   

                                             address  \
0      5943 Diana Gardens\nSouth Gregburgh, MT 88070   
1      

In [53]:
patient_data_gender = collection.find({"gender": "Female"}).limit(10)
# Create a list of dictionaries to store the extracted data
data_list = list(patient_data_gender)

# Create a Pandas DataFrame from the list of dictionaries
df_patient_gender = pd.DataFrame(data_list)

# Display the DataFrame
print(df_patient_gender)

                        _id  patient_id             name  gender  birth_date  \
0  671067a936b62531bc39057f      979020       Wendy Wise  Female        None   
1  671067a936b62531bc390580      108125  Jose Mcmahon MD  Female  1945-07-13   
2  671067a936b62531bc390583      855133      Keith Perez  Female        None   
3  671067a936b62531bc390585      721059       Ethan Odom  Female  1937-01-21   
4  671067a936b62531bc39058d        8688      Ronald Kane  Female  1960-06-07   
5  671067a936b62531bc390590      906950    Katrina Davis  Female        None   
6  671067a936b62531bc390592       44963       Daniel Day  Female        None   
7  671067a936b62531bc390594      703620       Cody Johns  Female  1955-02-20   
8  671067a936b62531bc390595      955357       Lynn Roach  Female  1972-07-01   
9  671067a936b62531bc39059b       56465      Marc Harris  Female  1937-06-09   

                                             address  \
0            055 Debra Landing\nPort Scott, VT 91484   
1      

# Task III - Data Transformations (30%)

In [55]:
df_patient

,_id,patient_id,name,gender,birth_date,address,admits
0,671067a936b62531bc39057e,948910,Chelsea Brady,None,1988-09-25,"5943 Diana Gardens\nSouth Gregburgh, MT 88070","[{'admit_reason': 'Fever', 'admit_date': '2020..."
1,671067a936b62531bc39057f,979020,Wendy Wise,Female,None,"055 Debra Landing\nPort Scott, VT 91484","[{'admit_reason': 'Respiratory issue', 'admit_..."
2,671067a936b62531bc390580,108125,Jose Mcmahon MD,Female,1945-07-13,Unit 6248 Box 8650\nDPO AE 58614,[]
3,671067a936b62531bc390581,883751,Maurice Scott,None,None,"185 Lane Gateway Apt. 682\nFergusonmouth, OR 1...","[{'admit_reason': 'Injury', 'admit_date': '202..."
4,671067a936b62531bc390582,324717,Dana Sullivan,None,None,"31945 Andrew Trail\nNorth Jonathan, VT 61173","[{'admit_reason': 'Digestive disorder', 'admit..."
5,671067a936b62531bc390583,855133,Keith Perez,Female,None,"11852 Chapman Loaf\nNorth Stanleyborough, PW 2...",[]
6,671067a936b62531bc390584,629891,Elizabeth Reyes,Male,None,"4246 Nolan Key Apt. 506\nRobertmouth, FM 41399","[{'admit_reason': 'Fever', 'admit_date': '2019..."
7,671067a936b62531bc390585,721059,Ethan Odom,Female,1937-01-21,"29570 Drake Garden\nSouth Jenniferland, CT 04245","[{'admit_reason': 'Chronic condition', 'admit_..."
8,671067a936b62531bc390586,386341,Cynthia Miller,None,None,"47932 Ronald Estate Suite 392\nEast Shawn, PR ...","[{'admit_reason': 'Infection', 'admit_date': '..."
9,671067a936b62531bc390587,629792,Darrell Perkins,Male,None,"88967 Sara Junctions\nAdamview, GA 60300","[{'admit_reason': 'Urinary tract infection', '..."


In [56]:
# Assuming df_patients is the DataFrame from Task II
df_patient = df_patient.drop('admits', axis=1, errors='ignore')

# Print the DataFrame after dropping the column
print(df_patient)

                        _id  patient_id             name  gender  birth_date  \
0  671067a936b62531bc39057e      948910    Chelsea Brady    None  1988-09-25   
1  671067a936b62531bc39057f      979020       Wendy Wise  Female        None   
2  671067a936b62531bc390580      108125  Jose Mcmahon MD  Female  1945-07-13   
3  671067a936b62531bc390581      883751    Maurice Scott    None        None   
4  671067a936b62531bc390582      324717    Dana Sullivan    None        None   
5  671067a936b62531bc390583      855133      Keith Perez  Female        None   
6  671067a936b62531bc390584      629891  Elizabeth Reyes    Male        None   
7  671067a936b62531bc390585      721059       Ethan Odom  Female  1937-01-21   
8  671067a936b62531bc390586      386341   Cynthia Miller    None        None   
9  671067a936b62531bc390587      629792  Darrell Perkins    Male        None   

                                             address  
0      5943 Diana Gardens\nSouth Gregburgh, MT 88070  
1        

In [57]:
import pandas as pd
from tqdm import tqdm
import sqlite3
import duckdb


%load_ext sql
%sql sqlite:///medication_database.db

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [58]:
con_medical_source = sqlite3.connect("/content/medication_database.db")
cur_medical_source = con_medical_source.cursor()
cur_medical_source

In [124]:
# Query the SQLite database and create a Pandas DataFra
query = "SELECT * FROM Medications"
df_medications = pd.read_sql_query(query, con_medical_source)

# Display the DataFrame
print(df_medications)

        patient_id      medication_name  dosage          frequency
0           198482         Atorvastatin      20         Once daily
1           198482           Gabapentin      83  Three times daily
2           198482         Atorvastatin      80         Once daily
3           198482          Simvastatin      11         Once daily
4           333846        Levothyroxine      11        Twice daily
...            ...                  ...     ...                ...
300455        9606          Montelukast      87  Three times daily
300456        9606        Levothyroxine      63  Three times daily
300457        9606             Losartan      87        Twice daily
300458        9606  Hydrochlorothiazide       6        Twice daily
300459        9606         Atorvastatin      14         Once daily

[300460 rows x 4 columns]


# Task IV - Load the Mongo data (15%)

In [88]:
# drop table (if exists)
cur_medical_source.execute('''
drop table if exists patient_admits
''')
con_medical_source.commit()

In [112]:
patient_data_2 = collection.find({}).limit(10)

# Create a list of dictionaries to store the extracted data
data_list = list(patient_data_2)

# Create a Pandas DataFrame from the list of dictionaries
df_patient_2 = pd.DataFrame(data_list)

# Display the DataFrame
print(df_patient_2)

                        _id  patient_id             name  gender  birth_date  \
0  671067a936b62531bc39057e      948910    Chelsea Brady    None  1988-09-25   
1  671067a936b62531bc39057f      979020       Wendy Wise  Female        None   
2  671067a936b62531bc390580      108125  Jose Mcmahon MD  Female  1945-07-13   
3  671067a936b62531bc390581      883751    Maurice Scott    None        None   
4  671067a936b62531bc390582      324717    Dana Sullivan    None        None   
5  671067a936b62531bc390583      855133      Keith Perez  Female        None   
6  671067a936b62531bc390584      629891  Elizabeth Reyes    Male        None   
7  671067a936b62531bc390585      721059       Ethan Odom  Female  1937-01-21   
8  671067a936b62531bc390586      386341   Cynthia Miller    None        None   
9  671067a936b62531bc390587      629792  Darrell Perkins    Male        None   

                                             address  \
0      5943 Diana Gardens\nSouth Gregburgh, MT 88070   
1      

In [113]:
# Create a list to store the data for the 'patient_admits' table
patient_admits_data = []

# Iterate through the DataFrame rows
for index, row in df_patient_2.iterrows():
  if 'patient_id' in row:  # Check if 'admits' exists for this patient
    patient_admits_data.append({
          'patient_id': str(row['patient_id']),
          'name': str(row['name']),
          'gender': str(row['gender']),
          'birthdate': str(row['birth_date']),
          'address': str(row['address']),
          'admits': str(row['admits'])
      })


# Connect to the SQLite database
con_medical_source = sqlite3.connect("/content/medication_database.db")
cur_medical_source = con_medical_source.cursor()

# Create the 'patient_admits' table if it doesn't exist
cur_medical_source.execute('''
    CREATE TABLE IF NOT EXISTS patient_admits (
        patient_id TEXT,
        name TEXT,
        gender TEXT,
        birthdate DATE,
        address TEXT,
        admits TEXT
    )
''')
con_medical_source.commit()

# Insert the data into the 'patient_admits' table
cur_medical_source.executemany('''
    INSERT INTO patient_admits (patient_id, name, gender, birthdate, address, admits)
    VALUES (?, ?, ?, ?, ?, ?)
''', [(d['patient_id'], d['name'], d['gender'], d['birthdate'], d['address'], d['admits']) for d in patient_admits_data])

con_medical_source.commit()

print("Patient admits data loaded into SQLite successfully.")

Patient admits data loaded into SQLite successfully.


In [114]:
patient_admits = pd.read_sql_query("SELECT * FROM patient_admits limit 10", con_medical_source)
patient_admits

,patient_id,name,gender,birthdate,address,admits
0,948910,Chelsea Brady,None,1988-09-25,"5943 Diana Gardens\nSouth Gregburgh, MT 88070","[{'admit_reason': 'Fever', 'admit_date': '2020..."
1,979020,Wendy Wise,Female,None,"055 Debra Landing\nPort Scott, VT 91484","[{'admit_reason': 'Respiratory issue', 'admit_..."
2,108125,Jose Mcmahon MD,Female,1945-07-13,Unit 6248 Box 8650\nDPO AE 58614,[]
3,883751,Maurice Scott,None,None,"185 Lane Gateway Apt. 682\nFergusonmouth, OR 1...","[{'admit_reason': 'Injury', 'admit_date': '202..."
4,324717,Dana Sullivan,None,None,"31945 Andrew Trail\nNorth Jonathan, VT 61173","[{'admit_reason': 'Digestive disorder', 'admit..."
5,855133,Keith Perez,Female,None,"11852 Chapman Loaf\nNorth Stanleyborough, PW 2...",[]
6,629891,Elizabeth Reyes,Male,None,"4246 Nolan Key Apt. 506\nRobertmouth, FM 41399","[{'admit_reason': 'Fever', 'admit_date': '2019..."
7,721059,Ethan Odom,Female,1937-01-21,"29570 Drake Garden\nSouth Jenniferland, CT 04245","[{'admit_reason': 'Chronic condition', 'admit_..."
8,386341,Cynthia Miller,None,None,"47932 Ronald Estate Suite 392\nEast Shawn, PR ...","[{'admit_reason': 'Infection', 'admit_date': '..."
9,629792,Darrell Perkins,Male,None,"88967 Sara Junctions\nAdamview, GA 60300","[{'admit_reason': 'Urinary tract infection', '..."


# Task V - Load the sqlite data (15%)

In [115]:
# drop table (if exists)
cur_medical_source.execute('''
drop table if exists patient_medications
''')
con_medical_source.commit()

In [125]:
medication_data = []

# Iterate through the DataFrame rows
for index, row in df_medications.iterrows():
    if 'patient_id' in row and 'medication_name' in row:  # Ensure 'patient_id' and 'medication_name' exist
        medication_data.append({
            'patient_id': str(row['patient_id']),  # Convert patient_id to string if needed
            'medication_name': str(row['medication_name']),  # Ensure medication_name is string
            'dosage': str(row['dosage']) if 'dosage' in row else None,  # Ensure dosage is string or set to None
            'frequency': str(row['frequency']) if 'frequency' in row else None,  # Ensure frequency is string or None
            'route': str(row['route']) if 'route' in row else None  # Ensure route is string or None
        })

# Create the 'patient_medications' table if it doesn't exist
cur_medical_source.execute('''
    CREATE TABLE IF NOT EXISTS patient_medications (
        patient_id TEXT,
        medication_name TEXT,
        dosage TEXT,
        frequency TEXT,
        route TEXT
    )
''')
con_medical_source.commit()

# Insert the data into the 'patient_medications' table
cur_medical_source.executemany('''
    INSERT INTO patient_medications (patient_id, medication_name, dosage, frequency, route)
    VALUES (?, ?, ?, ?, ?)
''', [(d['patient_id'], d['medication_name'], d['dosage'], d['frequency'], d['route']) for d in medication_data])

con_medical_source.commit()


print("Patient medications data loaded into SQLite successfully.")

Patient medications data loaded into SQLite successfully.


In [126]:
patient_medications = pd.read_sql_query("SELECT * FROM patient_medications limit 10", con_medical_source)
patient_medications

,patient_id,medication_name,dosage,frequency,route
0,198482,Atorvastatin,20,Once daily,None
1,198482,Gabapentin,83,Three times daily,None
2,198482,Atorvastatin,80,Once daily,None
3,198482,Simvastatin,11,Once daily,None
4,333846,Levothyroxine,11,Twice daily,None
5,333846,Fluticasone,48,Three times daily,None
6,333846,Prednisone,69,Three times daily,None
7,333846,Simvastatin,31,Three times daily,None
8,333846,Amlodipine,8,Once daily,None
9,401580,Azithromycin,51,Once daily,None


# Task VI - Create a sql view of your final patient_admit table (10%)

In [142]:
# drop table (if exists)
cur_medical_source.execute('''
drop view if exists patient_admit_view
''')
con_medical_source.commit()

In [143]:
%%sql
Create view patient_admit_view as
SELECT * FROM patient_admits LIMIT 10;

""


In [144]:
%%sql
SELECT * FROM patient_admit_view;

,patient_id,name,gender,birthdate,address,admits
0,948910,Chelsea Brady,None,1988-09-25,"5943 Diana Gardens\nSouth Gregburgh, MT 88070","[{'admit_reason': 'Fever', 'admit_date': '2020..."
1,979020,Wendy Wise,Female,None,"055 Debra Landing\nPort Scott, VT 91484","[{'admit_reason': 'Respiratory issue', 'admit_..."
2,108125,Jose Mcmahon MD,Female,1945-07-13,Unit 6248 Box 8650\nDPO AE 58614,[]
3,883751,Maurice Scott,None,None,"185 Lane Gateway Apt. 682\nFergusonmouth, OR 1...","[{'admit_reason': 'Injury', 'admit_date': '202..."
4,324717,Dana Sullivan,None,None,"31945 Andrew Trail\nNorth Jonathan, VT 61173","[{'admit_reason': 'Digestive disorder', 'admit..."
5,855133,Keith Perez,Female,None,"11852 Chapman Loaf\nNorth Stanleyborough, PW 2...",[]
6,629891,Elizabeth Reyes,Male,None,"4246 Nolan Key Apt. 506\nRobertmouth, FM 41399","[{'admit_reason': 'Fever', 'admit_date': '2019..."
7,721059,Ethan Odom,Female,1937-01-21,"29570 Drake Garden\nSouth Jenniferland, CT 04245","[{'admit_reason': 'Chronic condition', 'admit_..."
8,386341,Cynthia Miller,None,None,"47932 Ronald Estate Suite 392\nEast Shawn, PR ...","[{'admit_reason': 'Infection', 'admit_date': '..."
9,629792,Darrell Perkins,Male,None,"88967 Sara Junctions\nAdamview, GA 60300","[{'admit_reason': 'Urinary tract infection', '..."


In [145]:
# replace ###### with your file name
# make sure you have your google drive mounted.

!cp "/content/drive/MyDrive/Colab Notebooks/A7_Vu_Nguyen.ipynb" ./
!jupyter nbconvert --to html "A7_Vu_Nguyen.ipynb"

[NbConvertApp] Converting notebook A7_Vu_Nguyen.ipynb to html
[NbConvertApp] Writing 727377 bytes to A7_Vu_Nguyen.html
